In [1]:
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from ipywidgets import interact
import matplotlib.patches as patches


#### Change the directory to where the files are located" ####
day ='25'
directory = '../DVA/Data_Files/DVA_Day_Surveys/'

### Auxiliary functions

In [2]:
def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def Apply_Mask(mask, input_array): #TODO: This needs to be updated to implement all frequencies
    
    masked_copy = input_array.copy()
    mask_idx = np.where(mask == 1)
    masked_copy[mask_idx, :] = np.nan
    return(masked_copy)

### Read in the file listing azimuth scan start and stop times:

In [3]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(directory+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd


1052 2022-07-04T03:57:53.110 2022-07-04T04:15:55.184
2565 2022-07-04T04:20:37.406 2022-07-04T04:38:39.310
2566 2022-07-04T04:40:22.266 2022-07-04T04:58:24.003
1271 2022-07-04T05:03:42.716 2022-07-04T05:21:44.818
1272 2022-07-04T05:23:27.291 2022-07-04T05:41:29.292
2857 2022-07-04T05:48:35.269 2022-07-04T06:06:37.317
0124 2022-07-04T06:11:19.190 2022-07-04T06:29:21.101
1637 2022-07-04T06:34:03.920 2022-07-04T06:52:06.005
0342 2022-07-04T06:57:23.897 2022-07-04T07:15:25.930
1927 2022-07-04T07:21:55.804 2022-07-04T07:39:58.034
1928 2022-07-04T07:42:16.530 2022-07-04T08:00:18.319
2001 2022-07-04T08:03:48.814 2022-07-04T08:21:51.062
2002 2022-07-04T08:24:09.373 2022-07-04T08:42:11.545
2075 2022-07-04T08:45:41.821 2022-07-04T09:03:44.188
2148 2022-07-04T09:07:50.625 2022-07-04T09:25:52.832
2149 2022-07-04T09:27:35.229 2022-07-04T09:45:37.044
2150 2022-07-04T09:47:56.026 2022-07-04T10:05:57.828
2151 2022-07-04T10:07:40.752 2022-07-04T10:25:42.662
2152 2022-07-04T10:27:25.410 2022-07-04T10:45:

### Read in scan files and stich them together:

In [4]:
t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

scan0 = f"{scan_id[0]:04}"

# Use one of the scans to get the list of frequencies:
freq_channel_increment = 1 #TODO: I'll have to change this to 1 once I am sure I'm reading the data correctly

file = h5py.File(directory+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][::freq_channel_increment]/1e6
df = freq[1] - freq[0]

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])



# Loop through all the scans in the "scan_num" list:
for i in scan_id[0:5]:
    print(i)
    # select the file:
    file = h5py.File(directory+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']]) #This is a "mask" for noise regions 1 = noise 0=all good
    trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,::freq_channel_increment]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,::freq_channel_increment]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,::freq_channel_increment]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,::freq_channel_increment]],axis=0)

polarized = []
for i,j in zip(reRL_set,imRL_set):
    PI = np.sqrt((i**2)+(j**2))
    polarized.append(PI)
polarized_set = np.array(polarized)

noise_idx = np.array(np.where(noise_set == 1))
    
t_plt = Time(t_set, format='isot',scale='utc').mjd

#NOTE: Local change because Leo only has 5 scans on his computer
scan_id = [1052, 2565, 2566, 1271, 1272]

1052
<HDF5 file "dva_survey_raw_scan_1052.h5" (mode r)>
2565
<HDF5 file "dva_survey_raw_scan_2565.h5" (mode r)>
2566
<HDF5 file "dva_survey_raw_scan_2566.h5" (mode r)>
1271
<HDF5 file "dva_survey_raw_scan_1271.h5" (mode r)>
1272
<HDF5 file "dva_survey_raw_scan_1272.h5" (mode r)>


### Read in persistent RFI mask

In [5]:
i = 0
RFI_mask_idx = []
with open('../DVA/Data_Files/RFIpersist_mask.txt') as fp:
    for line in fp:
        if i>0: 
            RFI_mask_idx.append(int(line.split()[0]))
        i=i+1
RFI_mask_idx = np.array(RFI_mask_idx)

### Apply masks

In [6]:
polarized_set[noise_idx, :] = np.nan
polarized_set[:,RFI_mask_idx] = np.nan

LL_set[noise_idx, :] = np.nan
LL_set[:,RFI_mask_idx] = np.nan

RR_set[noise_idx, :] = np.nan
RR_set[:,RFI_mask_idx] = np.nan

In [7]:
def Combine_Into_RFI_Events(possible_RFI):
    Combined_RFI_events = [] 
    remaining_idxes = len(possible_RFI)

    current_idx = 0
    rfi_start = possible_RFI[current_idx]
    rfi_duration = 1

    while remaining_idxes >= 3:
        remaining_idxes = (len(possible_RFI) - current_idx)
        previous_idx = current_idx - 1
        idx_gap = (possible_RFI[current_idx] - possible_RFI[previous_idx])
        if(idx_gap <= 2):
            rfi_duration += 1
            current_idx += 1
        else:
            #Close current event
            Combined_RFI_events.append([rfi_start, rfi_duration])
            #Initiate next event starting on the current_idx
            rfi_duration = 1
            current_idx += 1
            rfi_start = possible_RFI[current_idx]

    return Combined_RFI_events #[idx, idx_duration]

# Spectrum Scan Functions

In [8]:
def RFI_Start_Found(time_array, spectrum_power_array, idx, slope_threshold):
    RFI_Start_Found = False
    spectrum_gradient = np.gradient(spectrum_power_array, 3)
    current_slope = spectrum_gradient[idx-1]
    next_slope = spectrum_gradient[idx]
    if((current_slope <= slope_threshold) and (next_slope >= slope_threshold)):
        RFI_Start_Found = True
        # print("RFI start information:\n - current slope:", current_slope,"next slope:", next_slope, "Slope threshold:", slope_threshold)
    return RFI_Start_Found


def RFI_End_Found(spectrum_power_array, rfi_end_idx, RFI_start_value):
    retVal = False
    if spectrum_power_array[rfi_end_idx] <= RFI_start_value:
        retVal = True
    return retVal

def RFI_Spectrum_Scan(time_array, spectrum_power_array, freq_idx, slope_threshold): #NOTE: This might be an issue if I have a nan value on my array
    scan_bandwidth = 10 #[MHz]
    start_idx = int(freq_idx - int((scan_bandwidth/df)/2))  #This changes the bandwidth from MHz to idxes
    end_idx = int(freq_idx + int((scan_bandwidth/df)/2))
    RFI_confirmed = False
    RFI_spectral_thickness = 0
    for rfi_start_idx in range(start_idx, end_idx):                                             #Scan over the entire bandwith interval
        if(RFI_Start_Found(time_array, spectrum_power_array, rfi_start_idx, slope_threshold)):     #If I found the starting pattern
            RFI_start_value = spectrum_power_array[rfi_start_idx]
            rfi_minimum_length = 10     #This exists to avoid the algorithm from fake crossing near the start due to fluctuations.
            for rfi_end_idx in range(rfi_start_idx + rfi_minimum_length, end_idx):                                   #Finish looking at the bandwidth interval looking for the end
                # if(Spectrum_RFI_End_Found(time_array, spectrum_power_array, rfi_end_idx, slope_threshold)):     #If I found the ending pattern          TODO: I have a problem in the end found algorithm
                if(RFI_End_Found(spectrum_power_array, rfi_end_idx, RFI_start_value)):     #If the RFI crosses below the starting value         TODO: I have a problem in the end found algorithm
                    RFI_confirmed = True      
                    break
                    # RFI_spectral_thickness = (rfi_end_idx - rfi_start_idx)
                else:
                    continue
            if(RFI_confirmed):                                                                          #and stop looping throught the interval for efficiency
                break
        else:
            rfi_end_idx = end_idx
    return RFI_confirmed, rfi_start_idx, (rfi_end_idx)#RFI_spectral_thickness

### RFI_Excision stuff below

In [9]:
def DVA_Find_Possible_RFI_Events(scan_chosen, freq_idx, baseline_multiplier, polarized_set):
    scan_id_plot = scan_chosen
    scan_idx = np.where(np.array(scan_id) == scan_id_plot)
    scan_duration_idx = np.where((t_plt>=scan_start_mjd[scan_idx]) & (t_plt<=scan_stop_mjd[scan_idx]))[0]

    scan_baseline = np.nanmedian(polarized_set[scan_duration_idx,freq_idx])
    scan_threshold = scan_baseline*baseline_multiplier

    print("np.shape(polarized_set[:, freq_idx])", np.shape(polarized_set[:, freq_idx]))
    possible_RFI_idxes = np.where(polarized_set[:, freq_idx] >= scan_threshold)
    print("np.shape(possible_RFI_idxes)", np.shape(possible_RFI_idxes))
    possible_RFI_events = Combine_Into_RFI_Events(possible_RFI_idxes[0])
    return possible_RFI_events  #Returns [time_idx, idx_duration]

In [10]:
def RFI_Verification(possible_RFI_events, freq_slope_threshold, event, freq_idx):
    rfi_confirmed = False
    for time_idx in range(possible_RFI_events[event][0], possible_RFI_events[event][0]+ possible_RFI_events[event][1]):
        event_verification_result = RFI_Spectrum_Scan(t_plt, polarized_set[time_idx, :], freq_idx, freq_slope_threshold)
        if event_verification_result[0]:
            rfi_confirmed = True
            break
    return rfi_confirmed, event_verification_result

def DVA_Find_Possible_Event_Start(scan, freq_idx, polarized_set, possible_RFI_events):
    scan_idx = np.where(np.array(scan_id) == scan)
    scan_duration_idx = np.where((t_plt>=scan_start_mjd[scan_idx]) & (t_plt<=scan_stop_mjd[scan_idx]))[0]
    scan_baseline = np.nanmedian(polarized_set[scan_duration_idx,freq_idx])
    start_event_idxes = []
    for event in range(0, len(possible_RFI_events)-1):
        start_found = False
        event_start_idx = possible_RFI_events[event][0]
        while not start_found:
            event_start_idx  = event_start_idx - 1
            if (polarized_set[event_start_idx, freq_idx] <= scan_baseline):
                start_event_idxes.append(event_start_idx)
                start_found = True
    return start_event_idxes

def DVA_Find_Possible_Event_End(scan, freq_idx, polarized_set, possible_RFI_events):
    scan_idx = np.where(np.array(scan_id) == scan)
    scan_duration_idx = np.where((t_plt>=scan_start_mjd[scan_idx]) & (t_plt<=scan_stop_mjd[scan_idx]))[0]
    scan_baseline = np.nanmedian(polarized_set[scan_duration_idx,freq_idx])
    end_event_idxes = []
    for event in range(0, len(possible_RFI_events)-1):
        end_found = False
        event_end_idx = possible_RFI_events[event][0] + possible_RFI_events[event][1]
        while not end_found:
            event_end_idx  = event_end_idx + 1
            if (polarized_set[event_end_idx, freq_idx] <= scan_baseline):
                end_event_idxes.append(event_end_idx)
                end_found = True
    return end_event_idxes

def RFI_Detection(scan, freq_slope_threshold, freq_chosen, baseline_multiplier):
    freq_idx = find_nearest_idx(freq, freq_chosen)
    freq_measured = freq[freq_idx]


    confirmed_RFI_results = []
    possible_RFI_events = DVA_Find_Possible_RFI_Events(scan, freq_idx, baseline_multiplier, polarized_set)
    possible_RFI_starts = DVA_Find_Possible_Event_Start(scan, freq_idx, polarized_set, possible_RFI_events)
    possible_RFI_ends = DVA_Find_Possible_Event_End(scan, freq_idx, polarized_set, possible_RFI_events)
    print("Number of possible RFI event:", len(possible_RFI_events))
    for event in range(0, len(possible_RFI_events)-1):
           # DETERMINE RFI REGION --------------------------------------------------------------------------------------------------------
        # t1_plt = possible_RFI_events[event][0]                                      #Start time     [idx]
        t1_plt = possible_RFI_starts[event]                                           #Start time     [idx]
        # t2_plt = possible_RFI_events[event][0] + possible_RFI_events[event][1]      #End time       [idx]
        t2_plt = possible_RFI_ends[event]                                             #End time       [idx]
        freq1 = freq_idx - 15                                        #Start freq     [idx]
        freq2 = freq_idx + 15                                        #End freq       [idx]
        
        confirmed_RFI_results.append([t1_plt, t2_plt, freq1, freq2])
        # rfi_confirmed, event_verification_result = RFI_Verification(possible_RFI_events, freq_slope_threshold, event, freq_idx)      
        # if rfi_confirmed:  
        #     # DETERMINE RFI REGION --------------------------------------------------------------------------------------------------------
        #     t1_plt = possible_RFI_events[event][0]                                      #Start time     [idx]
        #     t2_plt = possible_RFI_events[event][0] + possible_RFI_events[event][1]      #End time       [idx]
        #     freq1 = event_verification_result[1]                                        #Start freq     [idx]
        #     freq2 = event_verification_result[2]                                        #End freq       [idx]
            
        #     confirmed_RFI_results.append([t1_plt, t2_plt, freq1, freq2])

 
    print("Number of confirmed RFI regions:", len(confirmed_RFI_results))
    return confirmed_RFI_results

confirmed_RFI_results = RFI_Detection(scan = 1052, freq_slope_threshold = 1e5, freq_chosen = 844, baseline_multiplier = 3, run_freq_verification = True)
# interact(RFI_Detection(scan = 1045))


TypeError: RFI_Detection() got an unexpected keyword argument 'run_freq_verification'

# 2d Visualization plot

In [ ]:
def DVA_Singlescan_Time_Cross_Section(scan_chosen, freq_idx, duration, base_mult):
    fs = 14

    scan_id_plot = scan_chosen
    scan_idx = np.where(np.array(scan_id) == scan_id_plot)
    scan_duration_idx = np.where((t_plt>=scan_start_mjd[scan_idx]) & (t_plt<=scan_stop_mjd[scan_idx]))[0]

    scan_baseline = np.nanmedian(polarized_set[duration,freq_idx])

    possible_RFI_idxes = np.where(polarized_set[:, freq_idx] >= scan_baseline*3)
    polarized_set_masked = polarized_set.copy()
    polarized_set_masked[possible_RFI_idxes, :] = np.nan


    fig,axs1 = plt.subplots(1,1,figsize=(12.5,6))  
    axs1.plot(t_plt[duration], polarized_set[duration,freq_idx],color='red', linewidth = 0.5, label = "Possible RFI")
    axs1.plot(t_plt[duration], polarized_set_masked[duration,freq_idx],color='black', linewidth = 0.65, label = "Masked Polarized Set")


    axs1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs1.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs1.set_xlabel('Time (UTC)',fontsize=fs)
    axs1.set_ylabel("Power")
    axs1.set_xlim(t_plt[duration][0],t_plt[duration][-1])
    axs1.set_title("Polarized set of scan number ({:.0f}) with baseline {:.0f}".format(scan_chosen, scan_baseline))

    # axs1.axhline(abs_threshold,xmin=0,xmax=3,c="green",linewidth=1,zorder=0, label = "Absolute Threshold")
    axs1.axhline(scan_baseline*base_mult,xmin=0,xmax=3,c="purple",linewidth=1,zorder=0, label = "Relative Threshold")
    axs1.axhline(scan_baseline,xmin=0,xmax=3,c="red",linewidth=1,zorder=0, label = "Scan Baseline")
    axs1.set_ylim(ymax = np.nanmax(polarized_set[duration,freq_idx]), ymin=0)
    # axs1.set_ylim(ymax=1e6, ymin=0)
    axs1.tick_params(axis='both',labelsize=fs)   
    axs1.legend()

In [ ]:
def DVA_Plot_RFI(rfi_number, freq_chosen):
    freq_idx = find_nearest_idx(freq, freq_chosen)
    freq_measured = freq[freq_idx]
    freq_threshold = 1e5
    base_mult = 2
    confirmed_RFI_results = RFI_Detection(scan = 1052,freq_slope_threshold=freq_threshold, freq_chosen = freq_chosen, baseline_multiplier=base_mult, run_freq_verification = True)

    #Arbitraty constants for visualization   
    time_framing = 100
    freq_framing = 100
    time_buffer = 10
    freq_buffer = 10

    # DETERMINE RFI REGION --------------------------------------------------------------------------------------------------------
    t1_plt = confirmed_RFI_results[rfi_number][0]
    t2_plt = confirmed_RFI_results[rfi_number][1]
    freq1 = confirmed_RFI_results[rfi_number][2]
    freq2 = confirmed_RFI_results[rfi_number][3]
    print("RFI starting frequency: {:.2f}.\nRFI ending frequency: {:.2f}".format(freq1, freq2))

    # PLOTTING WATERFALL RFI ------------------------------------------------------------------------------------------------------

    RFI_duration_time = t_plt[t1_plt-time_framing: t2_plt+time_framing]
    RFI_freq_bandwith = freq[freq1 - freq_framing: freq2 + freq_framing]

    fig,axs = plt.subplots(1,2,figsize=(25,10)) 
    fs = 16

    # Create a Rectangle patch ---------------------------------------------------------------------------------------------------
    RFI_detected = patches.Rectangle((t_plt[t1_plt], freq[freq1]), np.abs(t_plt[t2_plt] - t_plt[t1_plt]), np.abs(freq[freq2] - freq[freq1]), linewidth=1.5, edgecolor='r', facecolor='none')
    RFI_detected_LL = patches.Rectangle((t_plt[t1_plt], freq[freq1]), np.abs(t_plt[t2_plt] - t_plt[t1_plt]), np.abs(freq[freq2] - freq[freq1]), linewidth=1.5, edgecolor='r', facecolor='none')
    # Add the patch to the Axes0
    axs[0].add_patch(RFI_detected)
    axs[1].add_patch(RFI_detected_LL)

    #Polarized Waterfall ---------------------------------------------------------------------------------------------------------------------------
    power_min = np.nanmedian((10.*np.log10(polarized_set[t1_plt-time_buffer: t2_plt+time_buffer,freq1-freq_buffer: freq2+freq_buffer].T)))
    power_max = np.nanmax((10.*np.log10(polarized_set[t1_plt-time_buffer: t2_plt+time_buffer,freq1-freq_buffer: freq2+freq_buffer].T)))

    im_pol = axs[0].imshow(10.*np.log10(polarized_set[t1_plt-time_framing: t2_plt+time_framing,freq1-freq_framing: freq2+freq_framing].T),aspect='auto',vmin=power_min,vmax=power_max,origin='lower',
            extent=[RFI_duration_time[0],RFI_duration_time[-1],RFI_freq_bandwith[0],RFI_freq_bandwith[-1]])

    divider = make_axes_locatable(axs[0])
    cax = divider.append_axes('right', size='2%', pad=0.05)
    cbar = fig.colorbar(im_pol, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=fs) 
    cbar.set_label('Power (dB)', fontsize=fs)

    axs[0].set_xlim(RFI_duration_time[0],RFI_duration_time[-1])
    axs[0].tick_params(axis='both', labelsize=fs)
    axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[0].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[0].set_xlabel('Time (UTC)',fontsize=fs)        
    axs[0].set_ylabel('Frequency (MHz)',fontsize=fs)
    axs[0].set_title('Polarized Set at Frequency {:.2f}'.format(freq_measured), size = fs)

    #LL Waterfall ---------------------------------------------------------------------------------------------------------------------------
    power_min = np.nanmedian((10.*np.log10(LL_set[t1_plt-time_buffer: t2_plt+time_buffer,freq1-freq_buffer: freq2+freq_buffer].T)))
    power_max = np.nanmax((10.*np.log10(LL_set[t1_plt-time_buffer: t2_plt+time_buffer,freq1-freq_buffer: freq2+freq_buffer].T)))

    im_LL = axs[1].imshow(10.*np.log10(LL_set[t1_plt-time_framing: t2_plt+time_framing,freq1-freq_framing: freq2+freq_framing].T),aspect='auto',vmin=power_min,vmax=power_max,origin='lower',
            extent=[RFI_duration_time[0],RFI_duration_time[-1],RFI_freq_bandwith[0],RFI_freq_bandwith[-1]])

    divider2 = make_axes_locatable(axs[1])
    cax2 = divider2.append_axes('right', size='2%', pad=0.05)
    cbar2 = fig.colorbar(im_LL, cax=cax2, orientation='vertical')
    cbar2.ax.tick_params(labelsize=fs) 
    cbar2.set_label('Power (dB)', fontsize=fs)

    axs[1].set_xlim(RFI_duration_time[0],RFI_duration_time[-1])
    axs[1].tick_params(axis='both', labelsize=fs)
    axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[1].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[1].set_xlabel('Time (UTC)',fontsize=fs)        
    axs[1].set_ylabel('Frequency (MHz)',fontsize=fs)
    axs[1].set_title('LL Set at Frequency {:.2f}'.format(freq_measured), size = fs)

    DVA_Singlescan_Time_Cross_Section(scan_chosen=1052, freq_idx = freq_idx, duration = np.arange(t1_plt-time_framing, t2_plt+time_framing))

    plt.hist(polarized_set[np.arange(t1_plt-time_framing, t2_plt+time_framing),freq_idx], bins = 'auto')
    plt.show()

# DVA_Plot_RFI(0, 835)
# interact(DVA_Plot_RFI, rfi_number = (0,34), freq_chosen = (343, 1031))
interact(DVA_Plot_RFI, rfi_number = (0,39), freq_chosen = 707)

interactive(children=(IntSlider(value=19, description='rfi_number', max=39), IntSlider(value=707, description=…

<function __main__.DVA_Plot_RFI(rfi_number, freq_chosen)>

In [ ]:
print(np.min(freq))
print(np.max(freq))

343.77083333333337
1031.1875


# Idea:

When I'm finding a possible RFI:
- Find the first time I'm crossing the baseline to the left -> Mark that as the start
- Find the first time I'm crossing the baseline to the right -> Mark that as the end

# Today

Get a bin view of a specific frequency channel.

Use that to see if I can determine that values far fom 1 stn deviation are off


In [ ]:
def plot_scan_histogram(polarized_set_masked, duration, freq_idx):
    polarized_set_masked[duration,freq_idx]
    hist, bin_edges = np.histogram(polarized_set_masked[duration,freq_idx], density=True)
    return hist



# Ideas

Maybe I could try implementing the "variance" compression method:
- check the variance of 5 points. if below threshold, compress 5 values with the same baseline value.
- Note: 5 is an arbitrary choice. I could even have a larger variance interval, and if the variance is high, I start dividing my large interval into smaller chunks looking for the high variance intervals